# Scraping and cleaning

The data is scraped from the website https://www.fahrrad-xxl.de/. We collect information about new bicycles. The output clean data contains the following columns:

<ul>
<li>"Preis" = price in Euro. (number) </li>
<li>"Gewicht" = weight in kg. (number) </li>
<li> "Gangzahl" = number of gears. (number)</li>
<li>"Bremse" = brake type: disc or rim. (string)</li>
<li> "Geschlecht" = male/female type. (string) </li>
<li> "Marke" = brand. (string) </li>
<li> "Rahmenmaterial" = material. (string) </li>
<li> "Federung" = suspension. (string) </li>
<li> "Farbe" = color. (string)</li>
<li> "DrehmomentinNm" = torque in Nm. (number) </li>
<li> "AkkukapazitätinWh" = accumulator capacity in Wh. (number) </li>
<li> "E-BikeMotorHersteller" = brand of the electro motor. (string) </li>
<li> "Modell" = name of the bicycle model. (string)</li>    
<li> "Elektro" = e-bike. (bool: True if yes, False if no)</li>
</ul> 

The indices are given in format "PiNj" where i is the number of page on the website and j is the number of the bicycle on the page. For example, the third bicycle on page 9 has the index "P9N3". Also, we save a small picture of the bicycle in a folder "bike_data" using the same format for naming, e.g., "P9N3.jpg".


## Outline:

1. [Scraping](#scr)<br>
2. [Cleaning](#cle)<br>

In [1]:
# Importing libraries.
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import os
import math

## 1. Scraping <a id="scr"></a>

In [3]:
# Choosing important features of a bicycle to be considered.
df = pd.DataFrame(columns=["Preis","Gewicht", "Schaltung", "Schaltwerk","Gangzahl", "Bremse", "Bremsevorne/hinten",
                           "Geschlecht", "Marke",
                           "Rahmenmaterial","Federung", "Rahmenform", "Beleuchtung", "Farbe", "DrehmomentinNm", 
                           "AkkukapazitätinWh", "Akku", "E-BikeMotorHersteller", "Modell"])
df

,Preis,Gewicht,Schaltung,Schaltwerk,Gangzahl,Bremse,Bremsevorne/hinten,Geschlecht,Marke,Rahmenmaterial,Federung,Rahmenform,Beleuchtung,Farbe,DrehmomentinNm,AkkukapazitätinWh,Akku,E-BikeMotorHersteller,Modell


In [4]:
# We go through a list of 87 pages each containing 48 bicycles. We enter each bicycle and collect some data
#corresponding to the columns of df.
for i in range(1,88,1): #going through pages
    source = requests.get(f"https://www.fahrrad-xxl.de/fahrraeder/seite/{i}/").text
    xxl = BeautifulSoup(source, "lxml")
    ware = xxl.find("div", class_="fxxl-warengruppe-detail-default")
    bikes = ware.find_all("a")
    k = 1
    time.sleep(1.0)
    for link in bikes: 
        #get image
        time.sleep(1.0)
        A = link.find("figure")
        image = A.find("img")
        a = image.get("data-src")
        b = image.get("src")
        if a != None:
            response = requests.get(a)
            file = open(f"bike_data/P{i}N{k}.jpg", "wb")
            file.write(response.content)
            file.close()
        else :
            response = requests.get(b)
            file = open(f"bike_data/P{i}N{k}.jpg", "wb")
            file.write(response.content)
            file.close()
        #enter bicycle
        source = requests.get(link.get("href")).text #reads website
        soup = BeautifulSoup(source, 'lxml') #starts soup object
        #read properties
        name = soup.find("h1", class_="fxxl-artikel-detail__product_name")
        preis = soup.find("div", class_="fxxl-artikel-detail__price fxxl-price")
        preis_rabat = soup.find("div", class_="fxxl-artikel-detail__price fxxl-price-with-strike-price")
        df.loc[f"P{i}N{k}","Modell"] = name.text
        if preis_rabat != None:
            df.loc[f"P{i}N{k}","Preis"] = preis_rabat.text
        else :
            df.loc[f"P{i}N{k}","Preis"] = preis.text
        items = soup.find_all('div', 
                      class_="fxxl-artikel-detail__grouping-properties-grid") #makes a list
        for it in items:
            a = it.find_all("div")
            for j in a:
                col = "".join(j.text.split())
                if (j.get("data-column") == "1") and (col in df.columns):
                    content = j.find_next_sibling("div") 
                    df.loc[f"P{i}N{k}", col] = content.text
        k += 1

In [5]:
df.head()

,Preis,Gewicht,Schaltung,Schaltwerk,Gangzahl,Bremse,Bremsevorne/hinten,Geschlecht,Marke,Rahmenmaterial,Federung,Rahmenform,Beleuchtung,Farbe,DrehmomentinNm,AkkukapazitätinWh,Akku,E-BikeMotorHersteller,Modell
P1N1,"1.799,99 €","23,30 kg",9-Gang Kettenschaltung,"Shimano Acera, 9-fach",09 - Gänge,"Shimano BR-MT200, hydraulische Scheibenbremse",NaN,Damen,CARVER,Aluminium,Mit Federung,Damen Sport,mit Beleuchtung,Weiß,40 Nm,"36 V - 11,2 Ah - 400 Wh",Bosch PowerPack 400 Performance,Bosch,Carver Tour-E LTD - 400 Wh - 2021 - 28 Zoll - ...
P1N2,"1.999,99 €",NaN,8-Gang Nabenschaltung (mit Rücktrittbremse),NaN,08 - Gänge,"Shimano BR-MT200, hydraulische Scheibenbremse",NaN,Damen,CARVER,Aluminium,Mit Federung,Tiefeinsteiger,mit Beleuchtung,Weiß,40 Nm,"36 V - 11,2 Ah - 400 Wh","Bosch PowerPack 400 Performance Line, Gepäckt...",Bosch,Carver E-Cityzen LTD RT - 400 Wh - 2021 - 28 Z...
P1N3,"1.799,99 €","23,20 kg",9-Gang Kettenschaltung,"Shimano Acera, 9-fach",09 - Gänge,"Shimano BR-MT200, hydraulische Scheibenbremse",NaN,Herren,CARVER,Aluminium,Mit Federung,Diamant,mit Beleuchtung,Grau,40 Nm,"36 V - 11,2 Ah - 400 Wh",Bosch PowerPack 400 Performance,Bosch,Carver Tour-E LTD - 400 Wh - 2021 - 28 Zoll - ...
P1N4,"1.899,99 €",NaN,8-Gang Nabenschaltung (mit Freilauf),NaN,07 - Gänge,"Shimano BR-MT200, hydraulische Scheibenbremse",NaN,Damen,CARVER,Aluminium,Mit Federung,Tiefeinsteiger,mit Beleuchtung,Grau,40 Nm,"36 V - 11,2 Ah - 400 Wh","Bosch PowerPack 400 Performance Line, Gepäckt...",Bosch,Carver E-Cityzen LTD FL - 400 Wh - 2021 - 28 Z...
P1N5,"2.449,- €","25,50 kg",9-Gang Kettenschaltung,"Shimano RD-M3100-SGS, 9-fach",09 - Gänge,NaN,"Shimano BR-MT200, hydraulische Scheibenbremse...",Herren,CUBE,Aluminium,Mit Federung,Diamant,mit Beleuchtung,NaN,50 Nm,"36 V - 17,4 Ah - 625 Wh",Bosch PowerTube 625,Bosch,Cube Touring Hybrid One 625 - 625 Wh - 2021 - ...


In [24]:
df.to_csv(r'bike_data/bikes.csv', index = True)

## 2. Cleaning <a id="cle"></a>
We write a pipeline that extracts the important information step by step from each column.

Transformations:

1. [Preis](#pre)<br>
2. [Gewicht](#gew)<br>
3. [Schaltwerk, Schaltung, Gangzahl](#sch)<br>
4. [Bremse, Bremsevorne/hinten](#bre)<br>
5. [Geschlecht, Marke, Rahmenmaterial, Federung, Rahmenform, Beleuchtung, Farbe](#ges)<br>
6. [DrehmomentinNm](#dre)<br>
7. [AkkukapazitätinWh, Akku](#akk)<br>
8. [E-BikeMotorHersteller, Modell](#e-b)<br>
9. [Pipeline](#pip)<br>

In [4]:
# Reading the data.
path = os.getcwd()
data_path = path + "/bike_data/bikes.csv"
df_0 = pd.read_csv(data_path)
df = df_0.set_index("Unnamed: 0")

In [5]:
df.describe()

,Preis,Gewicht,Schaltung,Schaltwerk,Gangzahl,Bremse,Bremsevorne/hinten,Geschlecht,Marke,Rahmenmaterial,Federung,Rahmenform,Beleuchtung,Farbe,DrehmomentinNm,AkkukapazitätinWh,Akku,E-BikeMotorHersteller,Modell
count,4166,2299,2917,2757,4049,560,3449,4165,4154,4122,4081,4076,4078,3280,1900,1948,1436,1916,4166
unique,560,713,123,832,22,204,783,6,101,7,2,12,2,18,16,47,313,20,4066
top,"2.999,- €","28,00 kg",12-Gang Kettenschaltung,"Shimano Deore, 10-fach",12 - Gänge,"Shimano BR-MT200, hydraulische Scheibenbremse...","Shimano BR-MT200, hydraulische Scheibenbremse...",Herren,CUBE,Aluminium,Mit Federung,Diamant,mit Beleuchtung,Schwarz,85 Nm,36 V - 14 Ah - 500 Wh,"Bosch Power Tube, Rahmenakku",Bosch,Lakes Rider 120 Street - 2021 - 20 Zoll - Diamant
freq,100,57,413,75,633,52,419,2002,372,3380,2659,1921,2337,1086,386,798,45,1177,3


### Preis <a id="pre"></a>

In [6]:
df.pivot_table(index=['Preis'], aggfunc='size')

Preis
 1.029,- €       1
 1.029,99 €      1
 1.039,90 €      1
 1.049,- €      30
 1.049,90 €      4
                ..
 949,90 €        2
 969,- €         1
 99,99 €         2
 999,- €        45
 999,99 €       27
Length: 560, dtype: int64

In [7]:
# Extract the price as a number.
# Input has form " 1.500,99 Eurosign " (thousand five hundred Euro and 99 cents). 
#Output should be 1500.00 (we drop the cents)
def price(string): 
    try: # if it is nan, we do nothing.
        math.isnan(string) 
        return np.nan
    except:
        a = string.split(" ")[1].split(",")[0] # we drop the cents.
        if len(a)>3: # two options 499,50 Euro and 2.499,50 Euro.
            return float(a)*1000
        else :
            return float(a)

class PreisToNumber(BaseEstimator, TransformerMixin):
    def fit(self,DF):
        return self
    def transform(self, DF):
        DF_new = DF.copy()
        A = DF["Preis"].apply(price)
        DF_new["Preis"] = A
        return DF_new
pr = PreisToNumber()
b = pr.fit_transform(df)
b["Preis"].dtypes #check

dtype('float64')

### Gewicht <a id="gew"></a>

In [8]:
df.pivot_table(index=['Gewicht'], aggfunc='size')

Gewicht
 1,8 kg           1
 1,9 kg           1
 10,0 kg          1
 10,00 kg         8
 10,02 kg         1
                 ..
 9.25 kg          1
 9.75 kg          1
 9.98 kg          1
 Ca. 2Kg          1
 ca. 28,00 kg     3
Length: 713, dtype: int64

In [9]:
# Extract the weight as a number.
# Input has form " 15,30 kg " or " ca. 15,30 kg" or " 15,30kg".
#Output should be 15.0 (we drop the dekagramms).
def weight(string): 
    try: # if it is nan, we do nothing.
        math.isnan(string) 
        return np.nan
    except:
        a = string.lower().split("kg")[0].split(" ")
        b = [x for x in a if x]
        c = b[len(b)-1]
        return float(c.split(",")[0])

class WeightToNumber(BaseEstimator, TransformerMixin):
    def fit(self,DF):
        return self
    def transform(self, DF):
        DF_new = DF.copy()
        A = DF["Gewicht"].apply(weight)
        DF_new["Gewicht"] = A
        return DF_new
wei = WeightToNumber()
B = wei.fit_transform(df)
B["Gewicht"].dtypes #check.

dtype('float64')

### Schaltwerk, Schaltung, Gangzahl <a id="sch"></a>

In [10]:
df.pivot_table(index=['Schaltwerk', 'Schaltung', 'Gangzahl'], aggfunc='size')

Schaltwerk                      Schaltung                    Gangzahl    
 1-Gang Nabe mit Rücktritt       1-Gang                       01 - Gang      1
 Dura_Ace 9100 Di2               Dura_Ace 9100 Di2            22 - Gänge     1
 Enviolo Cargo 380               stufenlose Nabenschaltung    Stufenlos      1
 GRX 810                         GRX 810                      11 - Gänge     1
 Microshift                      9-Gang Kettenschaltung       09 - Gänge     3
                                                                            ..
 Ultegra R8000                   Ultegra R8000                22 - Gänge     1
                                 Ultegra R8000 GS             22 - Gänge     1
 microSHIFT Advent X, 10-fach    10-Gang Kettenschaltung      10 - Gänge     1
 microSHIFT M26L, 8-fach         16-Gang Kettenschaltung      16 - Gänge     1
 microSHIFT M26L, 9-fach         18-Gang Kettenschaltung      18 - Gänge     3
Length: 654, dtype: int64

In [11]:
# It seems that the categories Schaltung and Schaltwerk are quite heterogeneosly and inconsistently filled in.
# For this reason, we drop them.
# For a more refined analysis, it could be possible to extract the following info about the transmission:
# brand and model, type of transmission.
#We keep only the Gangzahl category about the transmission properties.
df.pivot_table(index=['Gangzahl'], aggfunc='size')

Gangzahl
 01 - Gang       186
 02 - Gänge        4
 03 - Gänge      114
 05 - Gänge       95
 06 - Gänge       15
 07 - Gänge      403
 08 - Gänge      490
 09 - Gänge      240
 10 - Gänge      389
 11 - Gänge      302
 12 - Gänge      633
 14 - Gänge       17
 16 - Gänge       39
 18 - Gänge       51
 20 - Gänge      109
 21 - Gänge      125
 22 - Gänge      264
 24 - Gänge      171
 27 - Gänge      114
 30 - Gänge      143
 Singlespeed      41
 Stufenlos       104
dtype: int64

In [12]:
# Extract number of gears as a number.
#Input has form " 09 - Gaenge " or "Singlespeed" or "Stufenlos" (Singlespeed means there is 1 gear only,
# Stufenlos means there is continuous transition, we count this case as 1 gear too.)
#Output will be a number, e.g., 9.
def gear(string): 
    try:
        a = math.isnan(string) # if it is nan, we do nothing
        return string
    except:
        if (string == " Stufenlos ") or (string == " Singlespeed "):
            return 1.0
        else: 
            return float(string.split(" ")[1])
            
class GangzahlToNumber(BaseEstimator, TransformerMixin):
    def fit(self,DF):
        return self
    def transform(self, DF):
        DF_new = DF.copy()
        A = DF["Gangzahl"].apply(gear)
        DF_new["Gangzahl"] = A
        DF_new.drop(columns=["Schaltung", "Schaltwerk"], inplace=True)
        return DF_new
gang = GangzahlToNumber()
B = gang.fit_transform(df)
B["Gangzahl"].dtypes #check.

dtype('float64')

### Bremse, Bremsevorne/hinten <a id="bre"></a>

In [13]:
df.pivot_table(index=['Bremse', 'Bremsevorne/hinten'], aggfunc='size')

Bremse                                                        Bremsevorne/hinten                                                                                                       
 Dura-Ace Di2 Disc R9170                                       Shimano Dura Ace 9170, hydraulische Scheibenbremse/Shimano Dura Ace 9170, hydraulische Scheibenbremse                       1
 Magura HS11, hydraulische Felgenbremse                        Magura HS-11, hydraulische Felgenbremse/Magura HS-11, hydraulische Felgenbremse                                             1
 POWER, VBR -993A, V-Brake                                     Shimano BR-M365, Hydraulische Scheibenbremse                                                                                1
 Shimano BR-MT400 Hydraulische Scheibenbremse                  Shimano BR-MT400, hydraulische Scheibenbremse/Shimano BR-MT400, hydraulische Scheibenbremse                                 1
 Shimano MT201, hydraulische Scheibenbremse, Flat-Mount     

In [14]:
# Extract information about brake type. We will differentiate the following categories (by looking for keywords):
# disc (keyw: disc, scheibenbremse,...) = disc brakes; (we count roller brakes as disc brakes)
# rim (keyw: felgenbremse, v-brake, v-bremse, u-brake, u-bremse,...) = rim brakes.
kw_disc = ['disc', 'scheibenbremse', 'bosch abs', 'kolben', 'abs set', 'magura mt4', 'shimano mt-500',
          'tektro md', 'tektro hd', 'stromer', 'magura mt7', 'roller', 'shimano br-m', 'r9100','r8000',
          'ultegra 6800', 'r7000', 'shimano grx', 'xtr m9100', 'xtr m9000','shimano br-c3000f',
          'shimano slx', 'shimano m6000','tektro orion', 'shimano xt', 'shimano mt400','tektro t275',
          'shimano bl-mt'] 
kw_rim = ['felgenbremse', 'v-brake', 'v-bremse', 'u-brake', 'u-bremse', 'cantilever',
         'seitenzugbremse', 'caliper', 'magura hs11', 'shimano br-t4000', 'tiagra br-4700', 'tektro r',
         'tektro m', 'tektro c310', 'magura hs22']
def brake(string): 
    a = True
    for w in kw_disc:
        if w in string.lower():
            return 'disc'
    for w in kw_rim:
        if w in string.lower():
            return 'rim'
    return np.nan # if we cannot identify the brake type then nan
    # e.g., if it is written only ultegra we cannot decide since the ultegra group has both rim and disc versions        
            
class BrakeExtract(BaseEstimator, TransformerMixin):
    def fit(self,DF):
        return self
    def transform(self, DF):
        DF_new = DF.copy()
        A = DF_new["Bremse"].fillna(" ") + DF_new["Bremsevorne/hinten"].fillna(" ")
        B = A.apply(brake)
        DF_new["Bremse"] = B
        DF_new.drop(columns=['Bremsevorne/hinten'], inplace=True)
        return DF_new
bra = BrakeExtract()
C = bra.fit_transform(df)
C["Bremse"].describe() #check.

count     3898
unique       2
top       disc
freq      2816
Name: Bremse, dtype: object

In [17]:
# A short inspection of the bicycles where we cannot identify the brake type shows that these 
# contain a lot of unconventional bicycles such sa scooters, children bikes and transporters.
# We can use this fact to filter standard bicycles.
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time

# For the sake of keeping the size of the notebook we do not include these pictures.
# D = C[C['Bremse'].isna()]
# for i in D.index:
#     img=mpimg.imread('bike_data/'+i+'.jpg')
#     plt.imshow(img)
#     plt.show()

### Geschlecht, Marke, Rahmenmaterial, Federung, Rahmenform, Beleuchtung, Farbe <a id="ges"></a>

In [18]:
# The following categories are already clean. We do not touch them.
print(df.pivot_table(index=['Geschlecht'], aggfunc='size'),
      df.pivot_table(index=['Marke'], aggfunc='size'),
     df.pivot_table(index=['Rahmenmaterial'], aggfunc='size'),
     df.pivot_table(index=['Federung'], aggfunc='size'),
     df.pivot_table(index=['Rahmenform'], aggfunc='size'),
     df.pivot_table(index=['Beleuchtung'], aggfunc='size'),
     df.pivot_table(index=['Farbe'], aggfunc='size'))

Geschlecht
 Damen       1368
 Herren      2002
 Jungen       225
 Kinder       263
 Mädchen      100
 Unisex       207
dtype: int64 Marke
 ACADEMY          4
 BABBOE          15
 BACHTENKIR      42
 BELLINI         48
 BERGAMONT      172
               ... 
 VSF             31
 WIKE             1
 WILIER          21
 WINORA          98
 YUBA             2
Length: 101, dtype: int64 Rahmenmaterial
 Aluminium              3380
 Aluminium / Carbon        4
 Bambus                    1
 Carbon                  518
 Holz                      2
 Keine Angabe              4
 Stahl                   213
dtype: int64 Federung
 Mit Federung       2659
 Mit Starrgabel     1422
dtype: int64 Rahmenform
 BMX                  23
 Cruiser              24
 Damen Sport         507
 Diamant            1921
 Dirt                  7
 Doppelrohr           22
 Faltrahmen           70
 Fully               488
 Komfort              21
 Sonstiges            64
 Tiefeinsteiger      876
 Y-Form               53
dt

### DrehmomentinNm <a id="dre"></a>

In [19]:
df.pivot_table(index=['DrehmomentinNm'], aggfunc='size')

DrehmomentinNm
 120 Nm      11
 30 Nm        2
 35 Nm       27
 40 Nm      153
 44 Nm       12
 45 Nm        6
 50 Nm      299
 60 Nm      153
 63 Nm       65
 65 Nm      242
 70 Nm      232
 75 Nm      151
 80 Nm       81
 85 Nm      386
 90 Nm       78
 95 Nm        2
dtype: int64

In [20]:
# Extract the torque as a number.
# Input has form " 15 Nm ",
#Output should be 15.0.
# At this step we also add a column "Elektro" to mark e-bikes.
def torque(string): 
    try: # if it is nan, we do nothing.
        math.isnan(string) 
        return np.nan
    except:
        return float(string.split(" ")[1])

class TorqueToNumber(BaseEstimator, TransformerMixin):
    def fit(self,DF):
        return self
    def transform(self, DF):
        DF_new = DF.copy()
        A = DF["DrehmomentinNm"].apply(torque)
        DF_new["DrehmomentinNm"] = A
        DF_new["Elektro"] = df['DrehmomentinNm'].notnull() | df['Akku'].notnull() | df['AkkukapazitätinWh'].notnull() | df['E-BikeMotorHersteller'].notnull()
        return DF_new
tor = TorqueToNumber()
B = tor.fit_transform(df)
B["DrehmomentinNm"].dtypes #check.

dtype('float64')

### AkkukapazitätinWh, Akku <a id="akk"></a>

In [21]:
# We will combine the columns AkkukapazitätinWh and Akku to extract the number of Wh of the battery.
print(df.pivot_table(index=['AkkukapazitätinWh'], aggfunc='size'))
df.pivot_table(index=['Akku'], aggfunc='size')

AkkukapazitätinWh
 1000 Wh                       25
 1100 Wh                        3
 1250 Wh                        5
 208 Wh                         4
 216 Wh                         2
 26 V - 12,1 Ah - 317 Wh        1
 36 V - 10 Ah - 360 Wh          4
 36 V - 10,4 Ah - 374 Wh        1
 36 V - 10,5Ah - 378Wh          3
 36 V - 11 Ah - 396 Wh          2
 36 V - 11,2 Ah - 400 Wh      184
 36 V - 11,6 Ah - 418 Wh       20
 36 V - 12 Ah - 432 Wh          1
 36 V - 12,8 Ah - 460 Wh        3
 36 V - 13 Ah - 468 Wh          2
 36 V - 13,4 Ah - 482 Wh       11
 36 V - 13,8 Ah - 496 Wh       14
 36 V - 14 Ah - 500 Wh        798
 36 V - 14 Ah - 504 Wh        102
 36 V - 15 Ah - 540 Wh          8
 36 V - 17 Ah - 603 Wh          2
 36 V - 17 Ah - 612 Wh          2
 36 V - 17,2 Ah - 621 Wh        9
 36 V - 17,4 Ah - 625 Wh      434
 36 V - 17,5 Ah - 630 Wh      122
 36 V - 18 Ah - 650 Wh          2
 36 V - 18,6 Ah - 670 Wh        5
 36 V - 19,4 Ah - 700 Wh        5
 36 V - 20 Ah - 720 Wh        

Akku
 1000 Wh (Dual Batterie)                                     1
 2 x Bosch PowerPack 500 - Dual Battery, Rahmen-Akku         1
 2x 500 Wh                                                   1
 2x Bosch PowerPack 500 Performance, 36 V, 13,4 Ah/500Wh     2
 2x Bosch Powertube 500                                      2
                                                            ..
 Yamaha ML500, 36V, 14Ah                                     4
 Yamaha, Unterrohr Akku, 500?Wh                              8
 herausnehmbar, spritzwassergeschützt unter Trittbrett       1
 iNext Li-Ion                                                2
 voll integriert                                             1
Length: 313, dtype: int64

In [22]:
# Extract Wh of the battery as a number.
# Input has form "... 500 Wh ... " or "... 500Wh ..."
#Output should be 500.0.
def capacity(string): 
    a = string.split("Wh")[0].split(" ")
    b = [x for x in a if x]
    if len(b) == 0:
        return np.nan
    c = b[len(b)-1]
    try: #for the case that there is some info but not about Wh
        return float(c) 
    except:
        return np.nan

class CapacityToNumber(BaseEstimator, TransformerMixin):
    def fit(self,DF):
        return self
    def transform(self, DF):
        DF_new = DF.copy()
        A = DF["Akku"].fillna(" ") + DF["AkkukapazitätinWh"].fillna(" ")
        B = A.apply(capacity)
        DF_new["AkkukapazitätinWh"] = B
        DF_new.drop(columns=['Akku'], inplace=True)
        return DF_new
ak = CapacityToNumber()
B = ak.fit_transform(df)
B["AkkukapazitätinWh"].dtypes #check.

dtype('float64')

### E-BikeMotorHersteller <a id="e-b"></a>

In [23]:
#The columns E-BikeMotorHersteller is already clean. We do not touch it. Also, we do not change the column Modell.
print(df.pivot_table(index=['E-BikeMotorHersteller'], aggfunc='size'))

E-BikeMotorHersteller
 Bafang                 4
 Bosch               1177
 Brose                 22
 Coboc                  3
 Continental            6
 Ebikemotion           20
 Fazua                 19
 Giant SyncDrive      103
 Groove                 2
 Impulse                9
 Marquardt              1
 Panasonic             78
 Rocky Mountain         5
 SR Suntour            10
 Shimano Steps        197
 Specialized           41
 Stromer               27
 TQ                    11
 Xion                   1
 Yamaha               180
dtype: int64


### Pipeline <a id="pip"></a>

In [24]:
# Finally, we apply all the above transformations and save the clean data.
pipe = Pipeline([("preis_to_number", PreisToNumber()), ("weight_to_number", WeightToNumber()),
                ("gangzahl_to_number", GangzahlToNumber()), ("brake_extract", BrakeExtract()),
                 ("torque_to_number", TorqueToNumber()),("capacity_to_number", CapacityToNumber())])
df_clean = pipe.fit_transform(df)
df_clean.to_csv(r'bike_data/bikes_clean.csv', index = True)

In [27]:
df_clean.head()

,Preis,Gewicht,Gangzahl,Bremse,Geschlecht,Marke,Rahmenmaterial,Federung,Rahmenform,Beleuchtung,Farbe,DrehmomentinNm,AkkukapazitätinWh,E-BikeMotorHersteller,Modell,Elektro
Unnamed: 0,,,,,,,,,,,,,,,,
P1N1,1799.0,23.0,9.0,disc,Damen,CARVER,Aluminium,Mit Federung,Damen Sport,mit Beleuchtung,Weiß,40.0,400.0,Bosch,Carver Tour-E LTD - 400 Wh - 2021 - 28 Zoll - ...,True
P1N2,1999.0,NaN,8.0,disc,Damen,CARVER,Aluminium,Mit Federung,Tiefeinsteiger,mit Beleuchtung,Weiß,40.0,400.0,Bosch,Carver E-Cityzen LTD RT - 400 Wh - 2021 - 28 Z...,True
P1N3,1799.0,23.0,9.0,disc,Herren,CARVER,Aluminium,Mit Federung,Diamant,mit Beleuchtung,Grau,40.0,400.0,Bosch,Carver Tour-E LTD - 400 Wh - 2021 - 28 Zoll - ...,True
P1N4,1899.0,NaN,7.0,disc,Damen,CARVER,Aluminium,Mit Federung,Tiefeinsteiger,mit Beleuchtung,Grau,40.0,400.0,Bosch,Carver E-Cityzen LTD FL - 400 Wh - 2021 - 28 Z...,True
P1N5,2449.0,25.0,9.0,disc,Herren,CUBE,Aluminium,Mit Federung,Diamant,mit Beleuchtung,NaN,50.0,625.0,Bosch,Cube Touring Hybrid One 625 - 625 Wh - 2021 - ...,True
